In [10]:
data_root_path = 'D:\\jarondu\\Datasets\\Motion_X_one\\motion\\motion_generation\\vector_263\\animation\\animation'

file_name = 'Ways_to_Catch_360_clip1.npy'

In [11]:
import numpy as np
import os

data = np.load(os.path.join(data_root_path, file_name))
print(data.shape)

(89, 263)
